In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import os
import cv2
from ultralytics import YOLO


images_path = 'train folder path'
image_files = os.listdir(images_path)[:9]
fig, axes = plt.subplots(3, 3, figsize=(10, 10))
for ax, image in zip(axes.flatten(), image_files):
    img = plt.imread(os.path.join(images_path, image))
    ax.imshow(img)
    ax.axis('off')
plt.show()

In [ ]:
target_size = (640, 640)
train_images = []
for image in os.listdir(images_path):
    img = cv2.imread(os.path.join(images_path, image))
    img = cv2.resize(img, target_size)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    train_images.append(img)
train_images = np.array(train_images)

In [ ]:
# model = YOLO('yolo11n.yaml')  # build a new model from scratch
model = YOLO('yolo11s.pt')

In [ ]:
!yolo task=detect mode=train model=yolo11s.pt data="yaml file path" epochs=100 device=0 batch=32


In [ ]:
model = YOLO('/content/yolo11s.pt')
video_path = '/content/test.mp4'
cap = cv2.VideoCapture(video_path)
# Get video details
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(cv2.CAP_PROP_FPS))

out = cv2.VideoWriter('/content/output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO model on the frame
    results = model(frame)
    # Draw detections
    for r in results:
      # print(r.boxes)
      # break
        for box in r.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = box.conf[0]
            cls = int(box.cls[0])  # Get class ID

            # Assuming "falling person" class ID is 0
            if cls==0 and conf > 0.7:
                label = f"Fall {conf:.2f}"
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)  # Red box
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    # Write frame to output video
    out.write(frame)

    # Display the frame
    cv2.imshow(frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()